In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import pymc as pm
import arviz as az
from matplotlib import pyplot as plt
import networkx as nx
from scipy.stats import zscore

# install R and dagitty first: install.packages("dagitty")
import rpy2.robjects as ro
from rpy2.robjects.packages import importr

# Import R's dagitty
dagitty = importr('dagitty')

In [ ]:
# functions
def sim_weight(lengths, sd, beta_w):
    random_variation = np.random.normal(0, sd, len(lengths))  # random noise
    weights = beta_w*lengths + random_variation
    return weights

def sim_colour(lengths, sd, beta_c):
    random_variation = np.random.normal(0, sd, len(lengths))  # random noise
    colour = beta_c*lengths + random_variation
    return colour

def sim_predation(colours, weights, sd, beta_c, beta_w):
    random_variation = np.random.normal(0, sd, len(colours))  # random noise
    predation = beta_c*colours + beta_w*weights + random_variation
    return predation

def sim_reproduction(colours, weights, sd, beta_c, beta_w):
    random_variation = np.random.normal(0, sd, len(colours))  # random noise
    reproduction = beta_c*colours + beta_w*weights + random_variation
    return reproduction

def standardize(x):
    return zscore(x)

# Lab 3 - Underfitting, Overfitting, Predictive Accuracy and Model Comparison

**Author:** Arun Oakley-Cogan  
**Date:** 2025-10-06

In [ ]:
np.random.seed(123) 
# create frog data set
# number of frogs
n_frog = 75
# generate some lengths
lengths = np.random.normal(0, 1, n_frog)
# Length -> Weight
weights = sim_weight(lengths, 1, 1)
# Length -> Colour
colours = sim_colour(lengths, 1, -1)
# colour -> predation, weight -> predation
predation = sim_predation(colours, weights, sd=1, beta_c=1.5, beta_w=1)
# colour -> reproduction, weight->reproduction
reproduction = sim_reproduction(colours, weights, sd=1, beta_c=-1.5, beta_w=1)

frog_data = {
    'weights': standardize(weights),
    'lengths': standardize(lengths),
    'colours': standardize(colours),
    'reproduction': standardize(reproduction),
    'predation': standardize(predation)
}

In [ ]:
# draw dag
dag = dagitty.dagitty("dag{ Length -> Weight Length -> Colour Colour-> Reproduction Colour->Predation Weight->Predation Weight->Reproduction}")

# extract edges for Python plotting
edges_r = ro.r('function(d) { e <- edges(d); data.frame(from=e$v, to=e$w) }')(dag)
edges = [(str(edges_r[0][i]), str(edges_r[1][i])) for i in range(len(edges_r[0]))]

dag_nx = nx.DiGraph(edges)
# set coordinates
pos = {
    'Colour': (0, 0),
    'Length': (0, 1),
    'Reproduction': (1, 0),
    'Weight': (1, 1),
    'Predation': (0.5, 0.5)
}
# draw dag
nx.draw(dag_nx, pos, with_labels=True, node_color='lightblue',
        node_size=1500, arrowsize=20, arrows=True)
plt.title("Collider DAG: Predation as collider")
plt.show()

So far we have looked at the way statistical models can:

-   Describe the points in our data by fitting a model (linear regression)

-   Describe what function explains these points (causal inference)

We can also use statistical models to aid in *prediction*. What is going to be the next observation from the same process.

## What is LPPD?

**LPPD** stands for **Log Pointwise Predictive Density**. It's a measure of how well a model predicts data, and it's fundamental to modern Bayesian model comparison.

When we fit a Bayesian model, we get a **posterior distribution** of parameters.

LPPD can answer, "How well does this posterior distribution predict each observation in our data?"

For each data point $y_i$, LPPD calculates:

$$\text{lppd} = \sum_{i=1}^{N} \log \left( \frac{1}{S} \sum_{s=1}^{S} p(y_i | \theta_s) \right)$$

Where: 
- $N$ is the number of observations 
- $S$ is the number of posterior samples 
- $\theta_s$ is the $s^{th}$ sample from the posterior 
- $p(y_i | \theta_s)$ is the likelihood of observation $i$ given parameters $\theta_s$

For each observation, we average the likelihood across all posterior samples, take the log, and sum across all observations.

### Important for...

1.  **Prediction focus**: LPPD measures predictive accuracy, not just fit
2.  **Accounts for uncertainty**: Uses the full posterior distribution, not just point estimates
3.  **Foundation for model comparison**: LPPD is the basis for WAIC and LOO-CV
4.  **Detects overfitting**: Models that overfit will have high in-sample LPPD but poor out-of-sample LPPD

Let's use our simulated data to show it in action

In [ ]:
# Model 1: Intercept only
with pm.Model() as one_par:
    # priors
    alpha = pm.Normal("alpha", 0, 1)
    sigma = pm.Exponential("sigma", 1)
    
    # linear model
    mu = pm.Deterministic("mu", alpha)
    
    # data likelihood
    predation_obs = pm.Normal("predation", mu, sigma, observed=frog_data['predation'])
    
    one_par_idata = pm.sample(chains=4, idata_kwargs={"log_likelihood": True})

In [ ]:
# Model 2: Colour only
with pm.Model() as two_par:
    # priors
    alpha = pm.Normal("alpha", 0, 1)
    beta_c = pm.Normal("beta_c", 0, 1)
    sigma = pm.Exponential("sigma", 1)
    
    # linear model
    mu = pm.Deterministic("mu", alpha + beta_c*frog_data['colours'])
    
    # data likelihood
    predation_obs = pm.Normal("predation", mu, sigma, observed=frog_data['predation'])
    
    two_par_idata = pm.sample(chains=4, idata_kwargs={"log_likelihood": True})

In [ ]:
# Model 3: Colour + Length
with pm.Model() as three_par:
    # priors
    alpha = pm.Normal("alpha", 0, 1)
    beta_c = pm.Normal("beta_c", 0, 1)
    beta_l = pm.Normal("beta_l", 0, 1)
    sigma = pm.Exponential("sigma", 1)
    
    # linear model
    mu = pm.Deterministic("mu", alpha + beta_c*frog_data['colours'] + beta_l*frog_data['lengths'])
    
    # data likelihood
    predation_obs = pm.Normal("predation", mu, sigma, observed=frog_data['predation'])
    
    three_par_idata = pm.sample(chains=4, idata_kwargs={"log_likelihood": True})

In [ ]:
# Model 4: Colour + Length + Weight
with pm.Model() as four_par:
    # priors
    alpha = pm.Normal("alpha", 0, 1)
    beta_c = pm.Normal("beta_c", 0, 1)
    beta_l = pm.Normal("beta_l", 0, 1)
    beta_w = pm.Normal("beta_w", 0, 1)
    sigma = pm.Exponential("sigma", 1)
    
    # linear model
    mu = pm.Deterministic("mu", alpha + beta_c*frog_data['colours'] + beta_l*frog_data['lengths'] + beta_w*frog_data['weights'])
    
    # data likelihood
    predation_obs = pm.Normal("predation", mu, sigma, observed=frog_data['predation'])
    
    four_par_idata = pm.sample(chains=4, idata_kwargs={"log_likelihood": True})

In [ ]:
# Model 5: Colour + Length + Weight + Reproduction
with pm.Model() as five_par:
    # priors
    alpha = pm.Normal("alpha", 0, 1)
    beta_c = pm.Normal("beta_c", 0, 1)
    beta_l = pm.Normal("beta_l", 0, 1)
    beta_w = pm.Normal("beta_w", 0, 1)
    beta_r = pm.Normal("beta_r", 0, 1)
    sigma = pm.Exponential("sigma", 1)
    
    # linear model
    mu = pm.Deterministic("mu", alpha + beta_c*frog_data['colours'] + beta_l*frog_data['lengths'] + beta_w*frog_data['weights'] + beta_r*frog_data['reproduction'])
    
    # data likelihood
    predation_obs = pm.Normal("predation", mu, sigma, observed=frog_data['predation'])
    
    five_par_idata = pm.sample(chains=4, idata_kwargs={"log_likelihood": True})

In [ ]:
# Model 6: Polynomial (adding quadratic term)
frog_data['reproduction2'] = frog_data['reproduction']**2

with pm.Model() as poly_par:
    # priors
    alpha = pm.Normal("alpha", 0, 1)
    beta_c = pm.Normal("beta_c", 0, 1)
    beta_l = pm.Normal("beta_l", 0, 1)
    beta_w = pm.Normal("beta_w", 0, 1)
    beta_w2 = pm.Normal("beta_w2", 0, 1)
    beta_r = pm.Normal("beta_r", 0, 1)
    sigma = pm.Exponential("sigma", 1)
    
    # linear model
    mu = pm.Deterministic("mu", alpha + beta_c*frog_data['colours'] + beta_l*frog_data['lengths'] + beta_w*frog_data['weights'] + beta_r*frog_data['reproduction'] + beta_w2*frog_data['reproduction2'])
    
    # data likelihood
    predation_obs = pm.Normal("predation", mu, sigma, observed=frog_data['predation'])
    
    poly_par_idata = pm.sample(chains=4, idata_kwargs={"log_likelihood": True})

In [ ]:
# Extract LPPD for each model
lppd_p1 = az.loo(one_par_idata).elpd_loo
lppd_p2 = az.loo(two_par_idata).elpd_loo
lppd_p3 = az.loo(three_par_idata).elpd_loo
lppd_p4 = az.loo(four_par_idata).elpd_loo
lppd_p5 = az.loo(five_par_idata).elpd_loo
lppd_p6 = az.loo(poly_par_idata).elpd_loo

# Create comparison table
lppd_comparison = pd.DataFrame({
    'Model': ['M1: Intercept Only', 
              'M2: Colour Only', 
              'M3: Colour + Length', 
              'M4: Colour + Length + Weight',
              'M5: Colour + Length + Weight + Reproduction',
              'M6: Polynomial'
              ],
    'LPPD': [lppd_p1, lppd_p2, lppd_p3, lppd_p4, lppd_p5, lppd_p6],
    'Parameters': [2, 3, 4, 5, 6, 7]
})

print(lppd_comparison)

**Higher LPPD is better** - it means the model assigns higher probability density to the observed data.

Looking at our results:

1.  **M1 (Intercept Only)**: Poorest LPPD - ignores all predictors
2.  **M2 (Colour Only)**:
3.  **M3 (Colour + Length)**:
4.  **M4 (Colour + Length + Weight)**:
5.  **M5 (Colour + Length + Weight + Reproduction)**:
6.  **M6 (Polynomial)**: Highest LPPD YAY

The posterior distributions of our five parameter model do the best job at predicting our data.

## From LPPD to Model Comparison Metrics

LPPD is the foundation for:

### 1. WAIC (Widely Applicable Information Criterion)

$$\text{WAIC} = -2(\text{lppd} - p_{WAIC})$$

Where $p_{WAIC}$ is the effective number of parameters (penalty for overfitting).

### 2. LOO-CV (Leave-One-Out Cross-Validation)

Estimates out-of-sample LPPD by leaving out each observation and predicting it.

In [ ]:
# Compare models using WAIC and LOO
model_dict = {
    'one_par': one_par_idata,
    'two_par': two_par_idata,
    'three_par': three_par_idata,
    'four_par': four_par_idata,
    'five_par': five_par_idata,
    'poly_par': poly_par_idata
}

# WAIC comparison
waic_comparison = az.compare(model_dict, ic='waic')
print("WAIC Comparison:")
print(waic_comparison)

# LOO comparison
loo_comparison = az.compare(model_dict, ic='loo')
print("\nLOO Comparison:")
print(loo_comparison)

## Model Comparison vs Model Selection

One thing we can do with all of these tools is to perform *model selection*, which means choosing the model with the lowest PSIS/WAIC value. An issue with this idea is maximizing predictive accuracy is not the same is not the same as making inference based on causation. In fact, models that counfound causal inference can make better predictions.

Instead, we prefer a concept of *model comparison*, which uses multiple models to understand how different variables influence predictions and when used with a causal model and independencies help us infer causal relationships.

Let's go through a simulated example.

In [ ]:
# draw dag
dag = dagitty.dagitty("dag{ A-> C B->C}")

# extract edges for Python plotting
edges_r = ro.r('function(d) { e <- edges(d); data.frame(from=e$v, to=e$w) }')(dag)
edges = [(str(edges_r[0][i]), str(edges_r[1][i])) for i in range(len(edges_r[0]))]

dag_nx = nx.DiGraph(edges)
# Draw the DAG
pos = nx.spring_layout(dag_nx)
nx.draw(dag_nx, pos, with_labels=True, node_color='lightblue',
        node_size=1500, arrowsize=20, arrows=True)
plt.title("Collider DAG: A -> C <- B")
plt.show()

In [ ]:
# simulate our data
np.random.seed(123)
A = np.random.normal(0, 1, 100)
B = np.random.normal(0, 1, 100)

# effect sizes
beta_a = -1.5
beta_b = 1.5
C = np.random.normal(beta_a*A + beta_b*B, 1)

sim_data = {
    'A': standardize(A),
    'B': standardize(B),
    'C': standardize(C)
}

We want to see the effect of A on B

In [ ]:
# Model A effect on B
with pm.Model() as A_model:
    # priors
    alpha = pm.Normal("alpha", 0, 1)
    bA = pm.Normal("bA", 0, 1)
    sigma = pm.Exponential("sigma", 1)

    # linear model
    mu = pm.Deterministic("mu", alpha + bA*sim_data['A'])

    # data likelihood
    B_obs = pm.Normal("B", mu, sigma, observed=sim_data['B'])

    A_model_idata = pm.sample(chains=4, idata_kwargs={"log_likelihood": True})

In [ ]:
# Model A and C effect on B
with pm.Model() as AC_model:
    # priors
    alpha = pm.Normal("alpha", 0, 1)
    bC = pm.Normal("bC", 0, 1)
    bA = pm.Normal("bA", 0, 1)
    sigma = pm.Exponential("sigma", 1)

    # linear model
    mu = pm.Deterministic("mu", alpha + bC*sim_data['C'] + bA*sim_data['A'])

    # data likelihood
    B_obs = pm.Normal("B", mu, sigma, observed=sim_data['B'])

    AC_model_idata = pm.sample(chains=4, idata_kwargs={"log_likelihood": True})

Lets take a look at the effects of each model, we are expecting that A has no effect on B

In [ ]:
# Compare coefficient estimates
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

az.plot_forest(A_model_idata, var_names=['alpha', 'bA', 'sigma'], ax=axes[0])
axes[0].set_title('A Model (A -> B)')

az.plot_forest(AC_model_idata, var_names=['alpha', 'bA', 'bC', 'sigma'], ax=axes[1])
axes[1].set_title('AC Model (A + C -> B)')

plt.tight_layout()
plt.show()

Ok, so in our model with just A, we get what we expect, no effect of A on B, but when we introduce C (a collider), this drastically changes our estimate of the effect of A on B, because we have opened a back door path by including the collider.

What model do we use here? A_model or AC_model?

Now, lets see how they do at making predictions.

In [ ]:
# Compare models for prediction
collider_models = {
    'A_model': A_model_idata,
    'AC_model': AC_model_idata
}

loo_collider = az.compare(collider_models, ic='loo')
print("LOO Comparison for Collider Models:")
print(loo_collider)

The AC_model is the better model at predicting. Which model should we use now? Why is it so much better?

If we just went by the methods of model selection, we would pick AC_model, as it is better at predicting out-of-sample B's Why? because conditioning on the collider induces a statistical association, so adds to predictive accuracy. While the AC model is better at predicting it fails causally.

Lastly, lets go through these tables -

-   WAIC - SE: standard error of WAIC
-   dWAIC: difference between each models WAIC and the best WAIC in the set
-   dSE: standard error fro the best model
-   pWAIC: penalty term - these numbers are close to the number of parameters in the posterior